# Microsoft Power BI Cmdlets for Windows PowerShell and PowerShell Core
The following cmdlets are available on PowerShell Gallery and can be installed in an elevated PowerShell session:
- [Microsoft Power BI](https://docs.microsoft.com/en-us/powershell/power-bi/overview?view=powerbi-ps)
- [Data Gateway](https://docs.microsoft.com/en-us/powershell/module/datagateway/?view=datagateway-ps)

In [3]:
$modules = @("MicrosoftPowerBIMGMT", "DataGateway")

foreach ( $m in $modules ) 
{
    if (Get-Module -ListAvailable -Name $m -Verbose) {
        write-host "Module $m is already imported."
    } 
    else {
        Install-Module -Name $m -Force -Verbose -Scope CurrentUser
        Import-Module $m -Verbose
    }
}

VERBOSE: Populating RepositorySourceLocation property for module MicrosoftPowerBIMgmt.
Module MicrosoftPowerBIMGMT is already imported.
VERBOSE: Populating RepositorySourceLocation property for module DataGateway.
Module DataGateway is already imported.


# Available Cmdlets
Returns the available cmdlets from either the MicrosoftPowerBIMGMT or DataGateway module.

In [4]:
Get-Help *PowerBI* | Sort-Object Name | Format-Table -AutoSize


Name                                 Category Module                          Synopsis
----                                 -------- ------                          --------
Add-PowerBIDataset                   Cmdlet   MicrosoftPowerBIMgmt.Data       Adds a dataset to a …
Add-PowerBIEncryptionKey             Cmdlet   MicrosoftPowerBIMgmt.Admin      Adds an encryption k…
Add-PowerBIGroupUser                 Alias                                    Add-PowerBIWorkspace…
Add-PowerBIRow                       Cmdlet   MicrosoftPowerBIMgmt.Data       Adds rows to the spe…
Add-PowerBIServiceAccount            Alias                                    Connect-PowerBIServi…
Add-PowerBIWorkspaceUser             Cmdlet   MicrosoftPowerBIMgmt.Workspaces Gives permissions to…
Connect-PowerBIServiceAccount        Cmdlet   MicrosoftPowerBIMgmt.Profile    Log in to the Power …
Copy-PowerBIReport                   Cmdlet   MicrosoftPowerBIMgmt.Reports    Creates a copy of th…
Copy-PowerBITile         

In [5]:
Get-Help *DataGateway* | Sort-Object Name | Format-Table -AutoSize


Name                                    Category Module              Synopsis
----                                    -------- ------              --------
Add-DataGatewayCluster                  Cmdlet   DataGateway         Creates a new data gateway cl…
Add-DataGatewayClusterDatasourceUser    Cmdlet   DataGateway         Grants a user permission to u…
Add-DataGatewayClusterUser              Cmdlet   DataGateway         Add user and associated permi…
Connect-DataGatewayServiceAccount       Cmdlet   DataGateway.Profile Connect-DataGatewayServiceAcc…
Disconnect-DataGatewayServiceAccount    Cmdlet   DataGateway.Profile Disconnect-DataGatewayService…
Get-DataGatewayAccessToken              Cmdlet   DataGateway.Profile Get-DataGatewayAccessToken…
Get-DataGatewayCluster                  Cmdlet   DataGateway         Get gateway clusters for the …
Get-DataGatewayClusterDatasource        Cmdlet   DataGateway         Gets Power BI data sources fo…
Get-DataGatewayClusterDatasourceStatus  Cmdlet

# Connect to Power BI service
Login with Azure Active Directory user authentication or Service Principal credentials.

For Service Principal:
- Connect-PowerBIServiceAccount -ServicePrincipal -Credential (Get-Credential)

In [ ]:
Connect-PowerBIServiceAccount

# Scope

Indicates the scope of the call.
- -Scope Individual
    - Returns only workspaces assigned to the caller.
- -Scope Organization
    - Returns all workspaces within a tenant (must be an administrator to initiate).

In [ ]:
$scopeLevel = "Individual"

# Return a list of Power BI workspaces

Return a listing of the Power BI workspaces including Id, Name, IsReadOnly and IsOnDedicatedCapacity.

To list all workspaces, use the parameter -All

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -All
Write-Host "Total Number of Workspaces: $($pbiWorkspaces.Count)`n"

Get-PowerBIWorkspace -Scope Individual -First 5

# Return a list of Power BI datasets in a workspace

Return a listing of the Power BI datasets within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDatasets = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDatasets += Get-PowerBIDataset -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Dataset" } }
    
}

$pbiDatasets | Sort-Object -Property Id -Unique

# Return a list of Power BI dataflows in a workspace

Return a listing of the Power BI dataflows within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDataflows = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDataflows += Get-PowerBIDataflow -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Dataflow" } }
    
}

$pbiDataflows | Sort-Object -Property Id -Unique

# Return a list of Power BI datasources in a dataflow in a workspace

Return a listing of the Power BI datasources in a dataflow within a workspace including the additional attributes WorkspaceId, DataflowId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDataflowDatasources = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDataflows += Get-PowerBIDataflow -Scope $scopeLevel -WorkspaceId $workspace.Id

    ForEach ($dataflow in $pbiDataflows) {

        $pbiDataflowDatasources += Get-PowerBIDataflowDatasource -Scope $scopeLevel -WorkspaceId $workspace.Id -DataflowId $dataflow.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="DataflowId"; Expression={ $dataflow.Id } }, @{Name="Type"; Expression={ "Dataflow Datasource" } }

    }
    
}

$pbiDataflowDatasources | Sort-Object -Property Id -Unique

# Return a list of Power BI reports in a workspace

Return a listing of the Power BI reports within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiReports = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiReports += Get-PowerBIReport -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Report" } }
    
}

$pbiReports | Sort-Object -Property Id -Unique

# Return a list of Power BI dashboards in a workspace

Return a listing of the Power BI dasbhoards within a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDashboards = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDashboards += Get-PowerBIDashboard -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="Type"; Expression={ "Dashboard" } }
    
}

$pbiDashboards | Sort-Object -Property Id -Unique

# Return a list of Power BI tiles on a dashboard in a workspace

Return a listing of the Power BI tiles on a dashboard within a workspace including the additional attributes WorkspaceId, DashboardId and Type.



In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiTiles = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDashboards = Get-PowerBIDashboard -Scope $scopeLevel -WorkspaceId $workspace.Id

    ForEach ($dashboard in $pbiDashboards) {

        $pbiTiles += Get-PowerBITile -Scope $scopeLevel -WorkspaceId $workspace.Id -DashboardId $dashboard.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="DashboardId"; Expression={ $dashboard.Id } }, @{Name="Type"; Expression={ "Tile" } }

    }
    
}

$pbiTiles | Sort-Object -Property Id -Unique

# Return a list of the Power BI import state of a dataset in a workspace

Return a listing of the Power BI import state in a workspace including the additional attribute WorkspaceId.

- ImportState
    - Succeeded
    - Failed

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiImports = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiImports += Get-PowerBIImport -Scope $scopeLevel -WorkspaceId $workspace.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }
    
}

$pbiImports | Sort-Object -Property Id -Unique

# Return a list of the Power BI data sources of a dataset in a workspace

Return a listing of the Power BI data sources of a dataset in a workspace including the additional attributes WorkspaceId and Type.

In [ ]:
$pbiWorkspaces = Get-PowerBIWorkspace -Scope $scopeLevel -First 5

$pbiDatasets = @()

ForEach ($workspace in $pbiWorkspaces) {

    $pbiDashboards = Get-PowerBIDashboard -Scope $scopeLevel -WorkspaceId $workspace.Id

    ForEach ($dashboard in $pbiDashboards) {

        $pbiTiles += Get-PowerBITile -Scope $scopeLevel -WorkspaceId $workspace.Id -DashboardId $dashboard.Id | select *, @{Name="WorkspaceId"; Expression={ $workspace.Id } }, @{Name="DashboardId"; Expression={ $dashboard.Id } }, @{Name="Type"; Expression={ "Tile" } }

    }
    
}

$pbiTiles | Sort-Object -Property Id -Unique